# Serve Falcon 7b Finetuned with LoRA using SageMaker LMI DLC

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

---

In this example we will demonstrate how to take a Falcon 7b model fine-tuned via the Low Rank Adapter (LoRA) method and deploy it for inference using SageMaker Large Model Inference (LMI) DLCs. We will use the DeepSpeed DLC for optimized inference. This is a natural extension to the blogpost that demonstrates how to use SageMaker to fine tune the Falcon 40b using QLoRA (Quantized LoRA). Please see the [blogpost](https://aws.amazon.com/blogs/machine-learning/interactively-fine-tune-falcon-40b-and-other-llms-on-amazon-sagemaker-studio-notebooks-using-qlora/) for more information.

We use the smaller 7b model in this example as it is quicker to deploy, and can be deployed to smaller, more available instances.

## Setup

Installs the dependencies required to package the model and run inferences using Amazon SageMaker. Update SageMaker, boto3 etc

In [ ]:
!pip install sagemaker boto3 --upgrade  --quiet

In [ ]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
from sagemaker.utils import name_from_base

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix_deepspeed = "hf-large-model-djl-/code_falcon7b_lora/deepspeed"  # folder within bucket where code artifact will go

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

## Create SageMaker Model Artifacts

The process of creating model artifacts for a LoRA fine-tuned model is very similar to the process for creating artifacts for a standard model to use with the LMI DLCs. For this notebook, we will be using [this LoRA checkpoint available on the huggingface hub](https://huggingface.co/dfurman/falcon-7b-openassistant-peft). If you have followed the blogpost for fine-tuning the Falcon 40b model, or have LoRA checkpoints for a different model, you can substitute those here. We will demonstrate two ways for creating model artifacts depending on whether you are deploying a fine-tuned model available from the HuggingFace hub, or a fine-tuned model with local artifacts.

In [ ]:
!rm -rf code_falcon7b_lora
!mkdir -p code_falcon7b_lora

### Option 1: Create a serving.properties for a LoRA checkpoint available on the hub

In [ ]:
%%writefile code_falcon7b_lora/serving.properties
engine=DeepSpeed
# built in default handler. handles model conversion from LoRA to deepspeed compatible
option.entryPoint=djl_python.deepspeed
# Pointer to LoRA checkpoints available on the huggingface hub
option.model_id=dfurman/falcon-7b-openassistant-peft
option.task=text-generation
option.tensor_parallel_degree=1
option.dtype=fp16
option.trust_remote_code=true
option.model_loading_timeout=1800


That is all you need for deploying a LoRA fine-tuned model available on the HuggingFace hub!

### Option 2: Create a serving.properties for a local LoRA checkpoint

If you have a LoRA checkpoint available locally that you want to deploy for inference, you will need to package the checkpoints as part of the code artifact that is uploaded to S3. The direcotry structure should look like this:

```
code_falcon7b_lora/
├── lora_checkpoint_dir
│   ├── adapter_config.json
│   ├── adapter_model.bin # can also be in safetensors format
├── (optional) base_model_artifacts
│   ├── *.bin # checkpoint files
│   ├── *.json # configuration files for model, tokenizer, generation etc.
├── serving.properties
```

If you have saved your LoRA checkpoints in a directory called `lora_checkpoint_dir`, the serving.properties file will look like this (uncommented, of course):

In [ ]:
# engine=DeepSpeed
# # Pointer to LoRA checkpoints available in a sub directory
# option.model_id=lora_checkpoint_dir
# # built in default handler. handles model conversion from LoRA to deepspeed compatible
# option.entryPoint=djl_python.deepspeed
# option.tensor_parallel_degree=1
# option.dtype=fp16
# option.trust_remote_code=true
# option.model_loading_timeout=1800

Note: For LoRA models generated by the HuggingFace Peft library, the adapter_config.json file specifies the base model that was used to train in a field called `base_model_name_or_path`. To use this model for inference, the LMI container expects the base model path relative to the model directory (in this example that would be the code_falcon7b_lora directory). If the directory is not found, the base model will be downloaded from HuggingFace hub at runtime. Please double check this path before deploying as some LoRA checkpoints on the hub specify a path that is local the environment used to fine-tune the model.

### Create requirements.txt

In [ ]:
%%writefile code_falcon7b_lora/requirements.txt
peft==0.4.0

## Create the Tarball and then upload to S3 location
Next, we will package our artifacts as `*.tar.gz` files for uploading to S3 for SageMaker to use for deployment

In [ ]:
!rm -f model.tar.gz
!rm -rf code_falcon7b_lora/.ipynb_checkpoints
!tar czvf model.tar.gz -C code_falcon7b_lora .
s3_code_artifact_deepspeed = sess.upload_data("model.tar.gz", bucket, s3_code_prefix_deepspeed)
print(f"S3 Code or Model tar for deepspeed uploaded to --- > {s3_code_artifact_deepspeed}")

## Define a serving container, SageMaker Model and SageMaker endpoint
Now that we have uploaded the model artifacts to S3, we can create a SageMaker endpoint.

### Define the serving container
Here we define the container to use for the model for inference. We will be using SageMaker's Large Model Inference(LMI) container using DeepSpeed.

In [ ]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.23.0"
)

### Create SageMaker model, endpoint configuration and endpoint.

In [ ]:
model_name_ds = name_from_base(f"falcon7b-lora-ds")
print(model_name_ds)

In [ ]:
create_model_response = sm_client.create_model(
    ModelName=model_name_ds,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact_deepspeed,
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
model_name = model_name_ds
print(f"Building EndpointConfig and Endpoint for: {model_name}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.8xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Run Inference

In this example we are running a LoRA fine-tuned version of the falcon-7b model, but the same steps can be used to deploy a LoRA fine-tuned version of falcon-40b (or any LoRA fine-tuned language model). Keep the following in mind if you are deploying the larger falcon-40b model.

Large models such as Falcon have very high accelerator memory footprint. Thus, a very large input payload or generating a large output can cause out of memory errors. We recommend using at least a `ml.g5.24xlarge` instance depending on the size of your LoRA checkpoints (such as applying LoRA to more weight tensors, using larger values for r).

If you find that increasing the input length or generation length leads to CUDA Out Of Memory errors, we recommend that you try one of the following solutions:
* Use 8 bit quantization. Set the `option.dtype` paramter to int8 in the `serving.properties` file. This will reduce the memory footprint of the model on the GPUs, allowing for larger input and generation sizes.
  * Using 8bit quantization may result in lower quality generated output.
* Deploy to an instance with more GPUs, and/or GPUs with more memory. The `ml.g5.48xlarge`, `ml.p4d.24xlarge`, and `ml.p4de.24xlarge` instances are all good options here.

You can pass additional generation arguments as part of the properties dictionary in the request (e.g. temperature, top_k etc.).

In [ ]:
%%time

prompt = """<human>: Create a list of 5 things to do in New York with a short description for each.
<bot>:
"""

data = {"inputs": prompt, "parameters": {"max_new_tokens": 512, "do_sample": True}}

response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(data),
    ContentType="application/json",
)

output = json.loads(response_model["Body"].read().decode("utf8"))
print(output[0]["generated_text"])

## Summary

In this example we showed how to take LoRA fine-tune checkpoints and deploy the full fine-tuned model on SageMaker using the DeepSpeed LMI DLC. This DLC will merge the LoRA weights with the base model weights, and then optimize the resulting model using tensor parallelism and fused CUDA kernels (if the model type is supported).

The same functionality is available in the FasterTransformer DLC. You can refer to the T5 LoRA inference example in this notebook: [Serve Flan-T5 Fine-Tuned with LoRA using SageMaker LMI DLC](https://github.com/aws/amazon-sagemaker-examples/tree/main/inference/generativeai/llm-workshop/lab11-lora-inference/flan-t5-lora.ipynb)

A unique feature of LoRA is that since the fine tuned model artifacts are separate from the base model, we can deploy a single endpoint with multiple adapters and switch between them at inference time. This functionality is demonstrated in this notebook: [Serve Multiple Fine-Tuned LoRA Adapters on a single endpoint using SageMaker LMI DLC](https://github.com/aws/amazon-sagemaker-examples/tree/main/inference/generativeai/llm-workshop/lab11-lora-inference/lora-multi-adapter.ipynb)

### Clean Up

In [ ]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/inference|generativeai|llm-workshop|lab11-lora-inference|falcon-7b-lora.ipynb)